# Assignment 4: Quantum Channel Classification
We reuse the Week 3 classifier to tag noisy channels quickly during calibration checks.

This week we focus on applying the saved classifier and showing that the workflow runs end to end.

**Task plan**
1. Explain why quick channel labels matter and list the workflow you will follow.
2. Load the helper code, pull in the trained estimator, and note any backup plan.
3. Rebuild the feature mapper from Kraus operators to Choi vectors.
4. Test the model on a small batch of synthetic channels and record observations.

## Background notes
- A quantum channel $\mathcal{E}$ is a completely positive, trace-preserving map. We write $\mathcal{E}(\rho) = \sum_k K_k \rho K_k^\dagger$ with $\sum_k K_k^\dagger K_k = I$ so that each Kraus operator $K_k$ captures one noise branch.
- The Choi matrix $C_{\mathcal{E}} = (\mathcal{E} \otimes I)(|\Phi^+\rangle\langle\Phi^+|)$ models how the channel acts on half of an entangled pair. Flattening its real and imaginary parts gives a steady feature vector.
- Classification is lighter than full tomography. We only emit labels like depolarising or amplitude damping, which keeps the calibration loop fast.

## Task 1 · Environment check
- Confirm qiskit, numpy, pandas, joblib, and scikit-learn import without errors.
- If anything is missing, run the pip cell below and log the command in your notes.
- Once the imports work, move to Task 2.

In [ ]:
# Log your environment status here once Task 1 is complete.


In [ ]:
# Install prerequisites if the kernel is missing a package.
# !pip install qiskit scikit-learn joblib pandas


## Task 2 · Import helper modules
- Run the cell below to pull in numpy, joblib, qiskit, pandas, and os.
- Keep the imports in one place so later tasks stay consistent.

In [ ]:
import numpy as np
import joblib
from sklearn.dummy import DummyClassifier
from qiskit.quantum_info import Kraus, Choi
import pandas as pd
import os


## Task 3 · Build a calibration-time classifier
- Implement a lightweight classifier that maps Choi features to channel labels without relying on saved artefacts.
- Keep the training code inside the provided function so reviewers can see your modelling choices.
- You may reuse utilities from earlier assignments (data loaders, feature encoders) as long as they are imported inside the function.

In [ ]:
def build_channel_classifier():
    """
    TODO: Train and return a classifier that distinguishes the channel families used in this exercise.

    Suggested steps:
        1. Create or load a labelled dataset of Choi features (can reuse helpers from Assignment 3).
        2. Split into train/validation sets to tune hyperparameters if needed.
        3. Fit a simple baseline model (e.g., logistic regression, random forest) and report key metrics via prints.
        4. Return the trained estimator so downstream cells can call `predict`.
    """
    raise NotImplementedError("Implement the calibration-time classifier for Task 3.")

model = build_channel_classifier()

## Task 4 · Build channel features
- Regenerate the Kraus operators you used during training or adapt them for this demo.
- Ensure `channel_to_feature` outputs the same ordering the model expects (real part first, imaginary part second).

In [ ]:
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

def depolarizing_kraus(p):
    k0 = np.sqrt(1 - p) * I
    k1 = np.sqrt(p/3) * X
    k2 = np.sqrt(p/3) * Y
    k3 = np.sqrt(p/3) * Z
    return Kraus([k0, k1, k2, k3])

def amplitude_damping_kraus(gamma):
    k0 = np.array([[1, 0], [0, np.sqrt(1-gamma)]], dtype=complex)
    k1 = np.array([[0, np.sqrt(gamma)], [0, 0]], dtype=complex)
    return Kraus([k0, k1])

def channel_to_feature(channel):
    choi = Choi(channel).data
    feat = np.concatenate([choi.real.flatten(), choi.imag.flatten()])
    return feat


## Task 5 · Classify sample channels
- Build a small list of synthetic channels, convert them with `channel_to_feature`, and stack the results in `X`.
- Use the loaded model to predict labels and review the DataFrame for any surprising cases.

In [ ]:
channels = [
    ('depolarizing_p0.1', depolarizing_kraus(0.1)),
    ('depolarizing_p0.5', depolarizing_kraus(0.5)),
    ('amp_damp_0.1', amplitude_damping_kraus(0.1)),
    ('amp_damp_0.5', amplitude_damping_kraus(0.5)),
]

features = []
names = []
for name, ch in channels:
    f = channel_to_feature(ch)
    names.append(name)
    features.append(f)
X = np.vstack(features)

preds = model.predict(X)
df = pd.DataFrame({'channel': names, 'prediction': preds})
df


### Submission checklist
- Update `model_path` with the actual artifact you trained in Assignment 3 and note the load result.
- Mention any feature changes you make so the classifier stays compatible with production runs.
- Save this notebook with outputs after running Tasks 1–5 and add a short reflection in your report.